In [1]:
import random
import torch
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
import string
import numpy as np
import nltk

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
!unzip hseds-texts-2020.zip > /dev/null 

In [4]:
PATH_TO_TRAIN_DATA = 'train.csv'

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7


In [6]:
df = df.loc[(df['negative'] != 'nothing') | (df['positive'] != 'no positive')]
df = df.loc[(df['negative'] != 'no negative') |  (df['positive'] != 'no positive')]
df = df.loc[(df['negative'] != 'no negative') |  (df['positive'] != 'nothing')]
df = df.loc[(df['negative'] != 'nothing') |  (df['positive'] != 'nothing')]

In [9]:
!pip install pytorch_transformers

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from pytorch_transformers import RobertaTokenizer, RobertaForSequenceClassification
from pytorch_transformers import RobertaConfig
from tqdm import tqdm

In [8]:
df['final_preprocessing'] = df['negative'] + '. ' + df['positive'] #сделаем столбец где будем хранить негативные и позитивные отзывы вместе

set_score = list(set(df.score.tolist()))
dict_score = {set_score[item]:item for item in range(len(set_score))}
def change(score):
    return(dict_score[score['score']])
  
df['score_class'] = df.apply(change, axis=1)

df_train, df_test = train_test_split(df)

In [9]:
sentences_train = [sentence for sentence in df_train['final_preprocessing']]
labels_train = [item for item in df_train['score_class'].tolist()]

sentences_test = [sentence for sentence in df_test['final_preprocessing']]
labels_test = [item for item in df_test['score_class'].tolist()]

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base',add_special_tokens=True)# будем использовать базовые Берт, где слова имеют нижний регистр


In [11]:
MAX_LEN = 250
train_input = [tokenizer.encode(x,add_special_tokens=True) for x in sentences_train]

train_pos_pad = pad_sequence([torch.as_tensor(seq[:MAX_LEN]) for seq in train_input], 
                           batch_first=True)

train_masks = [[float(i>0) for i in seq] for seq in train_pos_pad]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [12]:
test_input = [tokenizer.encode(x, add_special_tokens=True) for x in sentences_test]
test_pos_pad = pad_sequence([torch.as_tensor(seq[:MAX_LEN]) for seq in test_input], 
                           batch_first=True)

validation_masks = [[float(i>0) for i in seq] for seq in test_pos_pad]

Token indices sequence length is longer than the specified maximum sequence length for this model (679 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors


In [13]:
train_inputs = torch.tensor(train_pos_pad)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(train_masks)


validation_inputs = torch.tensor(test_pos_pad)
validation_labels = torch.tensor(labels_test)
validation_masks = torch.tensor(validation_masks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(
    train_data, shuffle=True,
    batch_size=32
)


validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(
    validation_data,
    batch_size=64
)

In [15]:
num_labels = len(df_train.score.unique())

In [16]:
config = RobertaConfig.from_pretrained("roberta-base", output_hidden_states=True, num_labels=num_labels)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config=config)

In [17]:
# функция перевода метки класса в score
dict_score_class = {item:set_score[item] for item in range(len(set_score))} #set_score = list(set(df.score.tolist())) (Напоминание)
def mae(pred, label):
    pred =  [dict_score_class[item] for item in pred]
    label = [dict_score_class[item] for item in label]
    return mean_absolute_error(pred, label)

In [18]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model.to(device).train()
    with tqdm(total=len(train_dataloader)) as pbar:
        for batch in train_dataloader:
            # добавляем батч для вычисления на GPU
            # Распаковываем данные из dataloader
            input_ids, input_mask, labels = batch
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            output = model.forward(input_ids, token_type_ids=None, attention_mask=input_mask)[0]
            _, predicted = torch.max(output, 1)
            
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(output.detach(), 1)
            accuracy_mae = mae(predicted.cpu().numpy(), labels.cpu().numpy())
            pbar.set_description('Loss: {:.4f}; Accuracy_MAE: {:.4f}'.format(loss.item(), accuracy_mae))    
            pbar.update(1)
            
def predict(model, val_dataloader, criterion, device="cuda:0"):
    model.to(device).eval()
    losses = []
    predicted_classes = []
    true_classes = []
    with tqdm(total=len(val_dataloader)) as pbar:
        with torch.no_grad():
            for batch in val_dataloader:
                
                input_ids, input_mask, labels = batch
                input_ids = input_ids.to(device)
                input_mask = input_mask.to(device)
                labels = labels.to(device)
                
                
                output = model.forward(input_ids, token_type_ids=None, attention_mask=input_mask)[0]
                _, predicted = torch.max(output, 1)
            
                loss = criterion(output, labels)
                losses.append(loss.item())
                _, predicted = torch.max(output.detach(), 1)
                predicted_classes.append(predicted)
                true_classes.append(labels)
                
                
                accuracy_mae = mae(predicted.cpu().numpy(), labels.cpu().numpy())
                pbar.set_description('Loss: {:.4f}; Accuracy_MAE: {:.4f}'.format(loss.item(), accuracy_mae))    
                pbar.update(1)
                
    predicted_classes = torch.cat(predicted_classes).detach().to('cpu').numpy()
    true_classes = torch.cat(true_classes).detach().to('cpu').numpy()
    return losses, predicted_classes, true_classes

def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    lrs = []
    for epoch in range(n_epochs):
        print('Learning rate: ', optimizer.param_groups[0]['lr'])
        print('Epoc:', epoch)
        train_one_epoch(model, train_dataloader, criterion, optimizer)
        print('Validation')
        losses, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
        print('Accuracy_MAE: ', mae(true_classes, predicted_classes))
        lrs.append(optimizer.param_groups[0]['lr'])
        scheduler.step()
    plt.plot(lrs)
    

In [19]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
learning_rate = 1e-05
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.1,step_size_up=5,mode="triangular2")
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001, last_epoch=-1)
n_epochs = 4

In [ ]:
learning_rate = 1e-05
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
n_epochs = 3

In [20]:
train(model, train_dataloader, validation_dataloader, criterion, optimizer, device, n_epochs, scheduler)

  0%|          | 0/2344 [00:00<?, ?it/s]

Learning rate:  1e-05
Epoc: 0


Loss: 2.3241; Accuracy_MAE: 0.9328:   0%|          | 1/391 [00:00<01:10,  5.54it/s]

Validation


  0%|          | 0/2344 [00:00<?, ?it/s]

...

Validation


  0%|          | 0/2344 [00:00<?, ?it/s]

Accuracy_MAE:  0.73834
Learning rate:  1.0000000000000002e-06
Epoc: 3


Loss: 1.7368; Accuracy_MAE: 0.4344%:   1%|          | 15/2344 [00:04<12:16,  3.16it/s]


KeyboardInterrupt: 

TEST

In [21]:
PATH_TO_TEST_DATA = 'test.csv'

In [22]:
df_test = pd.read_csv(PATH_TO_TEST_DATA)
df_test.head()

,review_id,negative,positive
0,00026f564b258ad5159aab07c357c4ca,Other than the location everything else was h...,Just the location
1,000278c73da08f4fcb857fcfe4ac6417,No UK TV but this was a minor point as we wer...,Great location very comfortable clean breakfa...
2,000404f843e756fe3b2a477dbefa5bd4,A tiny noisy room VERY deceptively photographed,The breakfast booked the preceding night but ...
3,000a66d32bcf305148d789ac156dd512,Noisy various electrical devices kicking in r...,Great location Nice bathroom
4,000bf1d8c5110701f459ffbedbf0d546,No Negative,Great location and friendly staff


In [23]:
df_test['final_preprocessing'] = df_test['negative'] + '. ' + df_test['positive']

In [24]:
sentences_test = [sentence for sentence in df_test['final_preprocessing']]

In [25]:
MAX_LEN = 250
test_input = [tokenizer.encode(x,add_special_tokens=True) for x in sentences_test]

test_pos_pad = pad_sequence([torch.as_tensor(seq[:MAX_LEN]) for seq in test_input], 
                           batch_first=True)

test_masks = [[float(i>0) for i in seq] for seq in test_pos_pad]

Token indices sequence length is longer than the specified maximum sequence length for this model (758 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (615 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (685 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (633 > 512). Running this sequence through the model will result in indexing errors


In [26]:
validation_inputs = torch.tensor(test_pos_pad)
validation_masks = torch.tensor(test_masks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [27]:
validation_data = TensorDataset(validation_inputs, validation_masks)
validation_dataloader = DataLoader(
    validation_data,
    batch_size=32
)

In [28]:
def predict(model, val_dataloader, device="cuda:0"):
    model.to(device).eval()
    losses = []
    predicted_classes = []
    true_classes = []
    valid_preds, valid_labels = [], []
    with tqdm(total=len(val_dataloader)) as pbar:
        with torch.no_grad():
            for step, batch in enumerate(val_dataloader):
                # добавляем батч для вычисления на GPU
                batch = tuple(t.to(device) for t in batch)

                # Распаковываем данные из dataloader
                input_ids, input_mask = batch

                logits = model(input_ids, token_type_ids=None, attention_mask=input_mask)

                logits = logits[0].detach().cpu().numpy()


                batch_preds = np.argmax(logits, axis=1)
    
                valid_preds.extend(batch_preds)

                pbar.set_description('Step: {:.4f}'.format(step))
                pbar.update(1)

    return valid_preds


In [29]:
predicted_classes = predict(model, validation_dataloader)


Step: 624.0000: 100%|██████████| 625/625 [00:58<00:00, 10.72it/s]


In [30]:
pred =  [dict_score_class[item] for item in predicted_classes]

In [31]:
df_test['score'] = pred

In [32]:
df_test = df_test[['review_id','score']]

In [33]:
df_test.to_csv('predict.csv', index=False) 

На Kaggle профиль alexserg98, MAE:0.74883